 Introduction

You are a Starbucks big data analyst ([that’s a real job!](https://www.forbes.com/sites/bernardmarr/2018/05/28/starbucks-using-big-data-analytics-and-artificial-intelligence-to-boost-performance/#130c7d765cdc)) looking to find the next store into a [Starbucks Reserve Roastery](https://www.businessinsider.com/starbucks-reserve-roastery-compared-regular-starbucks-2018-12#also-on-the-first-floor-was-the-main-coffee-bar-five-hourglass-like-units-hold-the-freshly-roasted-coffee-beans-that-are-used-in-each-order-the-selection-rotates-seasonally-5).  These roasteries are much larger than a typical Starbucks store and have several additional features, including various food and wine options, along with upscale lounge areas.  You'll investigate the demographics of various counties in the state of California, to determine potentially suitable locations.


In [1]:
import math
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim

import folium
from folium import Marker
from folium.plugins import MarkerCluster

In [2]:
root_path="/home/pliu/data_set/kaggle/geospatial/L04"

In [3]:
# Load and preview Starbucks locations in California
starbucks = pd.read_csv(f"{root_path}/starbucks_locations.csv")
starbucks.head()

,Store Number,Store Name,Address,City,Longitude,Latitude
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79


# Q1 Clean the data
Most of the stores have known (latitude, longitude) locations.  But, all the locations in the city of Berkeley are missing. You need to impute the latitude, and longitude column of this starbucks stores.

In [4]:
# How many rows in each column have missing values?
print(starbucks.isnull().sum())



Store Number    0
Store Name      0
Address         0
City            0
Longitude       5
Latitude        5
dtype: int64


In [5]:
# View rows with missing locations
rows_with_missing = starbucks[starbucks["City"]=="Berkeley"]
rows_with_missing

,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,NaN,NaN
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,NaN,NaN
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,NaN,NaN
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,NaN,NaN
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,NaN,NaN


In [6]:
geolocator = Nominatim(user_agent="kaggle_learn")
location = geolocator.geocode("2224 Shattuck Avenue Berkeley CA")

In [7]:
print(location.point)
print(location.address)

37 52m 7.8222s N, 122 16m 5.62872s W
Starbucks, 2224, Shattuck Avenue, Downtown Berkeley, Berkeley, Alameda County, CAL Fire Northern Region, California, 94701, United States


In [8]:
# this function takes an address then return the lat, and long
def my_geocoder(row):
    point = geolocator.geocode(row).point
    return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})

berkeley_locations = rows_with_missing.apply(lambda x: my_geocoder(x['Address']), axis=1)
starbucks.update(berkeley_locations)

# Q2 2) View Berkeley locations.

Let's take a look at the locations you just found.  Visualize the (latitude, longitude) locations in Berkeley in the OpenStreetMap style.

Then check if the shop are all in the `Berkeley` county on the map

In [9]:
m_2 = folium.Map(location=[37.88,-122.26], zoom_start=12)


for idx, row in starbucks[starbucks["City"]=="Berkeley"].iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row['Store Name']).add_to(m_2)

m_2

## Q3 Consolidate your data.

To make our analysis more meaningful, we will load a GeoDataFrame and three DataFrames:
- `ca_counties` containing the name, area (in square kilometers), and a unique id (in the "GEOID" column) for each county in the state of California.  The "geometry" column contains a polygon with county boundaries.
- `ca_pop` contains an estimate of the population of each county.
- `ca_high_earners` contains the number of households with an income of at least $150,000 per year.
- `ca_median_age` contains the median age for each county.

Use the four above data frame to create a new GeoDataFrame called `ca_stats`, and make sure it has 8 columns: **"GEOID", "name", "area_sqkm", "geometry", "population", "high_earners", and "median_age"**.  Also, make sure the CRS is set to `{'init': 'epsg:4326'}`.

In [15]:
ca_counties_path=f"{root_path}/CA_county_boundaries/CA_county_boundaries.shp"

In [16]:
ca_counties = gpd.read_file(ca_counties_path)
ca_counties.head()

,GEOID,name,area_sqkm,geometry
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6..."
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7..."
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822..."
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3..."
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360..."


In [12]:
ca_pop_path=f"{root_path}/CA_county_population.csv"
ca_pop = pd.read_csv(ca_pop_path, index_col="GEOID")
ca_pop.head()

,population
GEOID,
6001,1666753
6003,1101
6005,39383
6007,231256
6009,45602


In [13]:
ca_high_path=f"{root_path}/CA_county_high_earners.csv"
ca_high_earners = pd.read_csv(ca_high_path, index_col="GEOID")
ca_high_earners.head()

,high_earners
GEOID,
6001,145696
6003,30
6005,1220
6007,6860
6009,2046


In [14]:
ca_median_age_path=f"{root_path}/CA_county_median_age.csv"
ca_median_age = pd.read_csv(ca_median_age_path, index_col="GEOID")
ca_median_age.head()

,median_age
GEOID,
6001,37.3
6003,44.9
6005,50.6
6007,36.9
6009,51.6


In [17]:
# creat a new df by using join
# note if the first dataframe in the join is a geo dataframe, the result is a geo dataframe, if not, the result will be a dataframe. You can no longer do the geo operations

# step 1 join the three dataframe
cols_to_add = ca_pop.join([ca_high_earners, ca_median_age]).reset_index()

ca_stats = ca_counties.merge(cols_to_add, on="GEOID")

ca_stats.head()

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6...",2987,111,55.0
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7...",1540975,65768,35.9
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822...",446527,25231,33.7
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3...",45602,2046,51.6
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360...",850967,57121,37.5


In [18]:
# now change the crs code
ca_stats.crs={'init': 'epsg:4326'}

/home/pliu/git/GeoSpatial_analysis_python/venv/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [19]:
print(ca_stats.crs)

+init=epsg:4326 +type=crs


Now that we have all of the data in one place, it's much easier to calculate statistics that use a combination of columns.  Below code will create a "density" column with the population density.

In [20]:
ca_stats["density"] = ca_stats["population"] / ca_stats["area_sqkm"]
ca_stats.head()

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age,density
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6...",2987,111,55.0,1.198638
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7...",1540975,65768,35.9,598.376878
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822...",446527,25231,33.7,45.499825
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3...",45602,2046,51.6,16.980022
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360...",850967,57121,37.5,148.788107


## Q4 Which counties look promising?

Collapsing all of the information into a single GeoDataFrame also makes it much easier to select counties that meet specific criteria.

Use the next code cell to create a GeoDataFrame `sel_counties` that contains a subset of the rows (and all of the columns) from the `ca_stats` GeoDataFrame.  In particular, you should select counties where:
- there are at least 100,000 households making \$150,000 per year,
- the median age is less than 38.5, and
- the density of inhabitants is at least 285 (per square kilometer).

Additionally, selected counties should satisfy at least one of the following criteria:
- there are at least 500,000 households making \$150,000 per year,
- the median age is less than 35.5, or
- the density of inhabitants is at least 1400 (per square kilometer).